# Prep CONUS use-case.

This uses the GeoSpatialFabric v1.1. The idea with this notebook is to create the files necessary for running PRMS/NHM in pyonhm. The structure of the folder containing the data is as follows.  Here our root folder name is NHM_PRMS_CONUS_GF_1_1/.

```plaintext
NHM_PRMS_CONUS_GF_1_1/
    myparam.param
    control.default.bandit
    variable_info_new.json
    daily/
        input/
        output/
        restart/
    forecast/
        input/
        output/
        restart/
    GIS/

Where two key files from the bandit processed subset are manually placed in the root of the directory: myparam.param and control.default.bandit. Additionally the file variable_info_new.json, a json dictionary used for converting prms/nhm output to netcdf files. This file contains strings used to define variables, and conversion factors, and was developed by Steve Markstrom and used here.  The structure above is created manually and in the following steps we provide some additional processing to create additional files used in the pyonhm workflows.

In [2]:
import geopandas as gpd
import requests
import pandas as pd
import rioxarray
from pathlib import Path
import numpy as np
import zipfile
import sciencebasepy
import requests

## Get GFv1.1 fabric from sciencebase
- This will take a while...  Took 79 minutes 

In [3]:
fabric_path = Path("./GFv1.1.gdb.zip")
if not fabric_path.exists():
    sb = sciencebasepy.SbSession()
    sb.download_file(
        url="https://www.sciencebase.gov/catalog/file/get/5e29d1a0e4b0a79317cf7f63?f=__disk__90%2Fa5%2F04%2F90a50402567bd99e854bbb87669060dd1bdf5814",
        local_filename="GFv1.1.gdb.zip",
        progress_bar=True)


This will be the base directory you constructued from the directions above.

In [4]:
root_dir = '/mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/'

In [5]:
root_path = Path(root_dir)
daily_path = root_path / "daily"
daily_path.mkdir(exist_ok=True)
daily_path = root_path / "daily/input"
daily_path.mkdir(exist_ok=True)
daily_path = root_path / "daily/restart"
daily_path.mkdir(exist_ok=True)
daily_path = root_path / "daily/output"
daily_path.mkdir(exist_ok=True)
frcst_path = root_path / "forecast/"
frcst_path.mkdir(exist_ok=True)
frcst_path = root_path / "forecast/input"
frcst_path.mkdir(exist_ok=True)
frcst_path = root_path / "forecast/restart"
frcst_path.mkdir(exist_ok=True)
frcst_path = root_path / "forecast/output"
frcst_path.mkdir(exist_ok=True)
gis_path = root_path / "forecast/output"
gis_path.mkdir(exist_ok=True)

In [6]:
def read_param_values(dir, field):
    filename = Path(dir) / "myparam.param"
    values = []  # List to store the elevation values
    start_collecting = False  # Flag to start collecting values
    count_values = 0  # To count the values read
    
    with open(filename, 'r') as file:
        for line in file:
            stripped_line = line.strip()  # Remove any leading/trailing whitespace
            
            if stripped_line == field:
                # Skip the next four lines after 'hru_elev'
                for _ in range(4):
                    next(file)
                start_collecting = True  # Set flag to start collecting after skipping
                continue
            
            if stripped_line == '####' and start_collecting:
                break  # Stop reading if we hit #### after starting to collect
            
            if start_collecting:
                try:
                    # Convert the line to a float and add to the list
                    value = float(stripped_line)
                    values.append(value)
                    count_values += 1
                except ValueError:
                    continue  # If conversion fails, skip the line
        print(f"Read {count_values} {field} values")
    return np.array(values)

## Clean up hru shapefile and export as model_nhru.shp

In [31]:
gdf = gpd.read_file("GFv1.1.gdb.zip", layer="nhru_v1_1_simp")
# gdf = gpd.read_file("/mnt/c/Users/rmcd/OneDrive/OneDrive - DOI/Documents/data/GFV1.1.gdb", layer="nhru_v1_1_simp")
gdf

,nhru_v1_1,hru_segment_v1_1,nhm_id,hru_id_nat,Version,Shape_Length,Shape_Area,Change,geometry
0,76127,40038,76128,76128,1,80441.423292,1.881188e+08,-0.017302,"MULTIPOLYGON (((-105544.567 804074.976, -10541..."
1,76147,40038,76148,76148,1,53413.506290,4.418597e+07,0.054540,"MULTIPOLYGON (((-97185.217 806355.005, -97154...."
2,76170,40021,76171,76171,1,54988.828358,7.338919e+07,0.018316,"MULTIPOLYGON (((-105894.643 815045.861, -10570..."
3,76172,40019,76173,76173,1,31915.213081,3.629082e+07,0.049861,"MULTIPOLYGON (((-106762.641 815925.020, -10675..."
4,76181,40019,76182,76182,1,27390.023934,1.567346e+07,0.154732,"MULTIPOLYGON (((-109785.311 816675.047, -10978..."
...,...,...,...,...,...,...,...,...,...
114953,57964,31028,57965,57965,1,117117.944712,2.550804e+08,0.002392,"MULTIPOLYGON (((-294975.252 2728035.068, -2950..."
114954,64080,28886,64081,64081,1,67362.397583,1.102741e+08,0.002877,"MULTIPOLYGON (((-1284135.000 2658485.000, -128..."
114955,64150,28866,64151,64151,1,80355.338777,1.884838e+08,0.022769,"MULTIPOLYGON (((-1347645.097 2651834.819, -134..."
114956,65633,31412,65634,65634,1,71022.681331,7.386700e+07,-0.005068,"MULTIPOLYGON (((-1017265.000 2869285.000, -101..."


GFv1.1 has ~ 9000 in-valid polygon geometries, gdptools fixes invalid polygons when calculating the weights but we'll fix them here anyway.

In [32]:
# Find invalid geometries
invalid_geometries = gdf[~gdf.is_valid]
print(f"Found {len(invalid_geometries)} invalid geometries.")

# Correct invalid geometries by explicitly targeting the original DataFrame
gdf.loc[~gdf.is_valid, 'geometry'] = gdf.loc[~gdf.is_valid, 'geometry'].apply(lambda geom: geom.buffer(0))

# Re-check for invalid geometries after the correction
invalid_geometries_after = gdf[~gdf.is_valid]
print(f"Found {len(invalid_geometries_after)} invalid geometries after correction.")

Found 9988 invalid geometries.
Found 0 invalid geometries after correction.


In [33]:
gdf.crs

<Projected CRS: ESRI:102039>
Name: USA_Contiguous_Albers_Equal_Area_Conic_USGS_version
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Albers Equal Area
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [34]:
gdf.columns

Index(['nhru_v1_1', 'hru_segment_v1_1', 'nhm_id', 'hru_id_nat', 'Version',
       'Shape_Length', 'Shape_Area', 'Change', 'geometry'],
      dtype='object')

## Step 2: Sort the DataFrame by 'nhru_v1_1', and rename to nhm_id
- Simplify file for size

In [35]:
ngdf = gdf[["nhru_v1_1", "geometry"]].sort_values(by="nhru_v1_1")
ngdf = ngdf.rename(columns={"nhru_v1_1": "nhm_id"})
ngdf = ngdf.reset_index(drop=True)
ngdf


,nhm_id,geometry
0,1,"POLYGON ((1866614.821 2232974.892, 1866644.832..."
1,2,"MULTIPOLYGON (((1865744.747 2237114.794, 18655..."
2,3,"MULTIPOLYGON (((1862295.252 2239274.962, 18622..."
3,4,"MULTIPOLYGON (((1863045.106 2239185.024, 18631..."
4,5,"MULTIPOLYGON (((1873934.837 2238314.743, 18739..."
...,...,...
114953,114954,"MULTIPOLYGON (((-1944885.000 3177795.000, -194..."
114954,114955,"MULTIPOLYGON (((2135385.000 2699714.735, 21354..."
114955,114956,"MULTIPOLYGON (((2062701.753 2646525.047, 20629..."
114956,114957,"MULTIPOLYGON (((-580455.225 2871974.899, -5803..."


In [36]:
ngdf.to_parquet(f"{root_dir}GIS/model_nhru.parquet")

## Step 3: Export "nhru_v1_1" values from myparam.param file to a file

In [15]:
nhm_id = read_param_values(root_dir, "nhm_id")
df = pd.DataFrame(nhm_id.astype(int))
df.to_csv(f'{root_dir}/nhm_id', index=False, header=False)

Read 114958 nhm_id values


## Step 4: Export "hru_lat.csv" "hru_lon.csv" to a file as read from the .param file.
These are used when converting hru results as .csv file to netcdf output.  The lat/lon give some spatial context to the
netcdf output files.

In [16]:
hru_lat = read_param_values(root_dir, "hru_lat")
df = pd.DataFrame(hru_lat)
df.to_csv(f'{root_dir}/hru_lat.csv', index=False, header=False)

hru_lon = read_param_values(root_dir, "hru_lon")
df = pd.DataFrame(hru_lon)
df.to_csv(f'{root_dir}/hru_lon.csv', index=False, header=False)

Read 114958 hru_lat values
Read 114958 hru_lon values


## 5 Get elevations - used for calculating humidity from cfsv2 data.

Elevation is required for humidity conversion so we take the elevation of the hru centroid.

In [17]:

elevations = read_param_values(root_dir, "hru_elev")
df = pd.DataFrame(elevations)
df.to_csv(f'{root_dir}elevation.csv', index=False, header=False)

Read 114958 hru_elev values


## 6 Get segment lat/lon.  
The param db does not contain hru_lon generally and not hru_lat if there is no stream temp solved.  So rather than
using the db to extract those values we need to get them from the geometry file.

In [18]:
gdf_seg = gpd.read_file("GFv1.1.gdb.zip", layer="nsegment_v1_1")
gdf_seg

,nsegment_v1_1,tosegment_v1_1,seg_id_nhm,Version,Shape_Length,geometry
0,1,10,1,1,25338.448515,MULTILINESTRING Z ((2101948.624 2876678.641 0....
1,2,5,2,1,28122.099979,MULTILINESTRING Z ((2167789.031 2829021.852 0....
2,3,4,3,1,36010.226095,MULTILINESTRING Z ((2131936.492 2865675.020 0....
3,4,14,4,1,7652.059834,MULTILINESTRING Z ((2151719.943 2849594.051 0....
4,5,14,5,1,7394.318591,MULTILINESTRING Z ((2155981.103 2842240.715 0....
...,...,...,...,...,...,...
58484,23754,23749,57453,1.1,34231.913128,MULTILINESTRING Z ((129793.551 2795623.027 0.0...
58485,23491,23357,57190,1.1,13892.472320,MULTILINESTRING Z ((-350068.189 2908499.924 0....
58486,23401,23505,57100,1.1,114449.245953,MULTILINESTRING Z ((-512215.528 3002734.106 0....
58487,23434,23408,57133,1.1,18221.727430,MULTILINESTRING Z ((-463677.806 2967339.542 0....


In [19]:
invalid_geometries = gdf_seg[~gdf_seg.is_valid]
print(f"Found {len(invalid_geometries)} invalid geometries.")

Found 0 invalid geometries.


In [20]:
gdf_seg.columns

Index(['nsegment_v1_1', 'tosegment_v1_1', 'seg_id_nhm', 'Version',
       'Shape_Length', 'geometry'],
      dtype='object')

In [21]:
gdf_sorted = gdf_seg.sort_values(by="nsegment_v1_1")
gdf_sorted['nsegment_v1_1'].to_csv(
    f'{root_dir}nhm_seg', index=False, header=False
)
centroids = gdf_sorted.to_crs(5070).geometry.centroid
centroids_ll = centroids.to_crs(4326)
lats = centroids_ll.y
lons = centroids_ll.x
lats.to_csv(f'{root_dir}seg_lat.csv', index=False, header=False)
lons.to_csv(f'{root_dir}seg_lon.csv', index=False, header=False)

## Part 7 Generate gridmet weights.

In [22]:
from gdptools import WeightGen
from gdptools import AggGen
from gdptools import ClimRCatData


In [37]:
gdf = gpd.read_parquet(root_dir + "GIS/model_nhru.parquet")
gdf

,nhm_id,geometry
0,1,"POLYGON ((1866614.821 2232974.892, 1866644.832..."
1,2,"MULTIPOLYGON (((1865744.747 2237114.794, 18655..."
2,3,"MULTIPOLYGON (((1862295.252 2239274.962, 18622..."
3,4,"MULTIPOLYGON (((1863045.106 2239185.024, 18631..."
4,5,"MULTIPOLYGON (((1873934.837 2238314.743, 18739..."
...,...,...
114953,114954,"MULTIPOLYGON (((-1944885.000 3177795.000, -194..."
114954,114955,"MULTIPOLYGON (((2135385.000 2699714.735, 21354..."
114955,114956,"MULTIPOLYGON (((2062701.753 2646525.047, 20629..."
114956,114957,"MULTIPOLYGON (((-580455.225 2871974.899, -5803..."


In [38]:
climater_cat = "https://github.com/mikejohnson51/climateR-catalogs/releases/download/March-2024/catalog.parquet"
cat = pd.read_parquet(climater_cat)
cat.head()

,id,asset,URL,type,varname,variable,description,units,model,ensemble,...,Y1,Yn,resX,resY,ncols,nrows,crs,toptobottom,tiled,dim_order
0,bcca,None,https://cida.usgs.gov/thredds/dodsC/cmip5_bcca...,opendap,BCCA_0-125deg_pr_day_ACCESS1-0_rcp45_r1i1p1,pr,Precipitation,mm/d,ACCESS1-0,r1i1p1,...,25.1875,52.8125,0.125,0.125,462.0,222.0,+proj=longlat +a=6378137 +f=0.0033528106647474...,True,T,TYX
1,bcca,None,https://cida.usgs.gov/thredds/dodsC/cmip5_bcca...,opendap,BCCA_0-125deg_pr_day_ACCESS1-0_rcp85_r1i1p1,pr,Precipitation,mm/d,ACCESS1-0,r1i1p1,...,25.1875,52.8125,0.125,0.125,462.0,222.0,+proj=longlat +a=6378137 +f=0.0033528106647474...,True,T,TYX
2,bcca,None,https://cida.usgs.gov/thredds/dodsC/cmip5_bcca...,opendap,BCCA_0-125deg_pr_day_bcc-csm1-1_rcp26_r1i1p1,pr,Precipitation,mm/d,bcc-csm1-1,r1i1p1,...,25.1875,52.8125,0.125,0.125,462.0,222.0,+proj=longlat +a=6378137 +f=0.0033528106647474...,True,T,TYX
3,bcca,None,https://cida.usgs.gov/thredds/dodsC/cmip5_bcca...,opendap,BCCA_0-125deg_pr_day_bcc-csm1-1_rcp45_r1i1p1,pr,Precipitation,mm/d,bcc-csm1-1,r1i1p1,...,25.1875,52.8125,0.125,0.125,462.0,222.0,+proj=longlat +a=6378137 +f=0.0033528106647474...,True,T,TYX
4,bcca,None,https://cida.usgs.gov/thredds/dodsC/cmip5_bcca...,opendap,BCCA_0-125deg_pr_day_bcc-csm1-1_rcp60_r1i1p1,pr,Precipitation,mm/d,bcc-csm1-1,r1i1p1,...,25.1875,52.8125,0.125,0.125,462.0,222.0,+proj=longlat +a=6378137 +f=0.0033528106647474...,True,T,TYX


In [39]:
_id = "gridmet"
_varname = "tmmx"

# an example query returns a pandas dataframe.
tc = cat.query(
    "id == @_id & variable == @_varname"
)
tc

,id,asset,URL,type,varname,variable,description,units,model,ensemble,...,Y1,Yn,resX,resY,ncols,nrows,crs,toptobottom,tiled,dim_order
13997,gridmet,None,http://thredds.northwestknowledge.net:8080/thr...,opendap,daily_maximum_temperature,tmmx,tmmx,K,None,None,...,49.4,25.066667,0.041667,0.041667,1386.0,585.0,+proj=longlat +a=6378137 +f=0.0033528106647474...,False,,TYX


In [40]:
# Create a dictionary of parameter dataframes for each variable - here for the weights we just need one variable
tvars = ["tmmx"]
cat_params = [cat.query("id == @_id & variable == @_var").to_dict(orient="records")[0] for _var in tvars]

cat_dict = dict(zip(tvars, cat_params))

# Output an example of the cat_param.json entry for "aet".
cat_dict.get("tmmx")

{'id': 'gridmet',
 'asset': None,
 'URL': 'http://thredds.northwestknowledge.net:8080/thredds/dodsC/agg_met_tmmx_1979_CurrentYear_CONUS.nc',
 'type': 'opendap',
 'varname': 'daily_maximum_temperature',
 'variable': 'tmmx',
 'description': 'tmmx',
 'units': 'K',
 'model': None,
 'ensemble': None,
 'scenario': None,
 'T_name': 'day',
 'duration': '1979-01-01/..',
 'interval': '1 days',
 'nT': nan,
 'X_name': 'lon',
 'Y_name': 'lat',
 'X1': -124.76666663333334,
 'Xn': -67.05833330000002,
 'Y1': 49.400000000000006,
 'Yn': 25.066666666666666,
 'resX': 0.041666666666666664,
 'resY': 0.04166666666666668,
 'ncols': 1386.0,
 'nrows': 585.0,
 'crs': '+proj=longlat +a=6378137 +f=0.00335281066474748 +pm=0 +no_defs',
 'toptobottom': False,
 'tiled': '',
 'dim_order': 'TYX'}

In [41]:
user_data = ClimRCatData(
    cat_dict=cat_dict,
    f_feature=gdf,
    id_feature='nhm_id',
    period=["1980-01-01", "1980-12-31"]
)

wght_gen = WeightGen(
    user_data=user_data,
    method="parallel",
    output_file=root_dir + "GIS/conus_weights.csv",
    weight_gen_crs=6931,
    jobs=4
)

wghts = wght_gen.calculate_weights(intersections=False)

Using parallel engine
grid cells generated in 95.0385 seconds
Data preparation finished in 96.4174 seconds
     - validating target polygons
     - fixing 1 invalid polygons.
     - validating source polygons
     - fixing 0 invalid polygons.
Validate polygons finished in 6.5765 seconds
     - reprojecting and validating source polygons
     - checking the source polygons for invalid polygons
     - checking source for empty polygons
     - reprojecting and validating target polygons
     - checking the target polygons for invalid polygons
     - validating reprojected target polygons
     - fixing 3 invalid polygons.
     - checking target for empty polygons
Reprojecting to: EPSG:6931 and validating polygons finished in 17.51 seconds
Weight gen finished in 68.2738 seconds


In [10]:
import os
import subprocess
prms_path = '/home/rmcd/prms_5.2.1_linux/bin/prms'
project_root = '/mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1'
project_output = Path('/mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/daily/output')
cbh_path = Path("/mnt/d/oNHM/gm_cbh")
# hard-coded by looking at bandit.control file 
start_time = "1979-06-01"
end_time = "2021-12-31"
os.chdir(project_root)
print(os.getcwd())
command = [
   prms_path,
   "-set", "start_time", start_time,
   "-set", "end_time", end_time,
   "-set", "humidity_day", cbh_path / "humidity.cbh",
   "-set", "precip_day", cbh_path / "prcp.cbh",
   "-set", "tmax_day", cbh_path / "tmax.cbh",
   "-set", "tmin_day", cbh_path / "tmin.cbh",
   "-set", "save_vars_to_file", "1",
   "-set", "var_save_file", f"{project_root}/daily/restart/{end_time}.restart",
   "-set", "nhruOutBaseFileName", project_output,
   "-set", "nsegmentOutBaseFileName", project_output,
   "-set", "model_output_file", project_output / "model.out",
   "-set", "param_file", f"{project_root}/myparam.param",
   "-C", f"{project_root}/control.default.bandit"
]
subprocess.run(command, check=True)

/mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1



                         U.S. Geological Survey
               Precipitation-Runoff Modeling System (PRMS)
                        Version 5.2.1 02/08/2022

        Process            Available Modules
--------------------------------------------------------------------
  Basin Definition: basin
    Cascading Flow: cascade
  Time Series Data: obs, water_use_read, dynamic_param_read
   Potet Solar Rad: soltab
  Temperature Dist: temp_1sta, temp_laps, temp_dist2, climate_hru,
                    temp_map
       Precip Dist: precip_1sta, precip_laps, precip_dist2,
                    climate_hru, precip_map
Temp & Precip Dist: xyz_dist, ide_dist
    Solar Rad Dist: ccsolrad, ddsolrad, climate_hru
Transpiration Dist: transp_tindex, climate_hru, transp_frost
      Potential ET: potet_hamon, potet_jh, potet_pan, climate_hru,
                    potet_hs, potet_pt, potet_pm, potet_pm_sta
      Interception: intcp
Snow & Glacr Dynam: snowcomp, glacr_melt
  


Control variable start_time set to 1979-06-01.


Control variable end_time set to 2021-12-31.


Control variable humidity_day set to /mnt/d/oNHM/gm_cbh/humidity.cbh.


Control variable precip_day set to /mnt/d/oNHM/gm_cbh/prcp.cbh.


Control variable tmax_day set to /mnt/d/oNHM/gm_cbh/tmax.cbh.


Control variable tmin_day set to /mnt/d/oNHM/gm_cbh/tmin.cbh.


Control variable save_vars_to_file set to 1.


Control variable var_save_file set to /mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/daily/restart/2021-12-31.restart.


Control variable nhruOutBaseFileName set to /mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/daily/output.


Control variable nsegmentOutBaseFileName set to /mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/daily/output.


Control variable model_output_file set to /mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/daily/output/model.out.


Control variable param_file set to /mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/myparam.param.



Transpiration Distribution      transp_tindex       2021-09-07
Potential Evapotranspiration    potet_jh            2021-08-13
Canopy Interception             intcp               2021-09-07
Snow Dynamics                   snowcomp            2021-09-15
Surface Runoff                  srunoff_smidx       2021-08-18
Soilzone Computations           soilzone            2021-09-08
Groundwater                     gwflow              2021-08-13
Streamflow Routing Init         routing             2021-08-13
Streamflow Routing              muskingum_mann      2021-08-13
Stream Temperature              stream_temp         2021-09-07
Output Summary                  basin_sum           2021-08-13


The parameter seg_length has been declared inconsistently in the modules stream_temp and routing. The minimum value has been declared as 0.001 and 1.0. The maximum value has been declared as 200000.0 and 100000.0.
 The parameter description has been set as "Bed slope of each segment"and "Surface slope of each segment." The minimum value has been declared as 0.0001 and 0.0000001.






Using Control File: /mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/control.default.bandit

Using Parameter File: /mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/myparam.param

Writing var_save_file: /mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/daily/restart/2021-12-31.restart

Writing PRMS Water Budget File: /mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/daily/output/model.out

WARNING, HRU: 914 soil_rechr_max < 0.00001, set to 0.00001

WARNING, HRU: 112485 soil_rechr_max < 0.00001, set to 0.00001

WARNING, HRU: 113708 soil_rechr_max < 0.00001, set to 0.00001



         set in the Parameter File. Module default values are being used.


 WARNING, seg_slope < 0.0000001, set to 0.0001         188   0.00000000    
 WARNING, seg_slope < 0.0000001, set to 0.0001        3822   0.00000000    
 WARNING, seg_slope < 0.0000001, set to 0.0001        5216   0.00000000    
 WARNING, seg_slope < 0.0000001, set to 0.0001        5671   0.00000000    
 WARNING, seg_slope < 0.0000001, set to 0.0001        7629   0.00000000    
 WARNING, seg_slope < 0.0000001, set to 0.0001        8675   0.00000000    
 WARNING, seg_slope < 0.0000001, set to 0.0001        9679   0.00000000    
 WARNING, seg_slope < 0.0000001, set to 0.0001       11463   0.00000000    
 WARNING, seg_slope < 0.0000001, set to 0.0001       15020   0.00000000    
 WARNING, seg_slope < 0.0000001, set to 0.0001       15760   0.00000000    
 WARNING, seg_slope < 0.0000001, set to 0.0001       15873   0.00000000    
 WARNING, seg_slope < 0.0000001, set to 0.0001       16437   0.00000000    
 WARNING, seg_slope < 0.0000001, set to 0.0001       16740   0.00000000    
 WARNING, se

CompletedProcess(args=['/home/rmcd/prms_5.2.1_linux/bin/prms', '-set', 'start_time', '1979-06-01', '-set', 'end_time', '2021-12-31', '-set', 'humidity_day', PosixPath('/mnt/d/oNHM/gm_cbh/humidity.cbh'), '-set', 'precip_day', PosixPath('/mnt/d/oNHM/gm_cbh/prcp.cbh'), '-set', 'tmax_day', PosixPath('/mnt/d/oNHM/gm_cbh/tmax.cbh'), '-set', 'tmin_day', PosixPath('/mnt/d/oNHM/gm_cbh/tmin.cbh'), '-set', 'save_vars_to_file', '1', '-set', 'var_save_file', '/mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/daily/restart/2021-12-31.restart', '-set', 'nhruOutBaseFileName', PosixPath('/mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/daily/output'), '-set', 'nsegmentOutBaseFileName', PosixPath('/mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/daily/output'), '-set', 'model_output_file', PosixPath('/mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/daily/output/model.out'), '-set', 'param_file', '/mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/myparam.param', '-C', '/mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/control.default.bandit'], returncode=0)

In [26]:
import os
import subprocess
prms_path = '/home/rmcd/prms_5.2.1_linux/bin/prms'
project_root = '/mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1'
cbh_path = Path("/mnt/d/oNHM/gm_cbh")
# hard-coded by looking at bandit.control file 
start_date = "2022-01-01"
end_date = "2022-01-04"
start_time = "2022,1,1,0,0,0"
end_time = "2022,1,4,0,0,0"
restart_time = "2021-12-31"
os.chdir(project_root)
command = [
   prms_path,
   "-set", "start_time", start_time,
   "-set", "end_time", end_time,
   "-set", "humidity_day", f"{project_root}/daily/input/humidity.cbh",
   "-set", "precip_day", f"{project_root}/daily/input/prcp.cbh",
   "-set", "tmax_day", f"{project_root}/daily/input/tmax.cbh",
   "-set", "tmin_day", f"{project_root}/daily/input/tmin.cbh",
   "-set", "init_vars_from_file", "1",
   "-set", "var_init_file", f"{project_root}/daily/restart/{restart_time}.restart",
   "-set", "save_vars_to_file", "1",
   "-set", "var_save_file", f"{project_root}/daily/restart/{end_date}.restart",
   "-set", "nhruOutBaseFileName", project_output,
   "-set", "nsegmentOutBaseFileName", project_output,
   "-set", "model_output_file", project_output / "model.out",
   "-set", "param_file", f"{project_root}/myparam.param",
   "-C", f"{project_root}/NHM.control"
]
subprocess.run(command, check=True)


Control variable start_time set to 2022,1,1,0,0,0.


Control variable end_time set to 2022,1,4,0,0,0.


Control variable humidity_day set to /mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/daily/input/humidity.cbh.


Control variable precip_day set to /mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/daily/input/prcp.cbh.


Control variable tmax_day set to /mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/daily/input/tmax.cbh.


Control variable tmin_day set to /mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/daily/input/tmin.cbh.


Control variable init_vars_from_file set to 1.


Control variable var_init_file set to /mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/daily/restart/2021-12-31.restart.


Control variable save_vars_to_file set to 1.


Control variable var_save_file set to /mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/daily/restart/2022-01-04.restart.


Control variable nhruOutBaseFileName set to /mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/daily/output.


Control variable nsegmentOutBaseFileName set to /mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/daily/output.


Control variable 




                         U.S. Geological Survey
               Precipitation-Runoff Modeling System (PRMS)
                        Version 5.2.1 02/08/2022

        Process            Available Modules
--------------------------------------------------------------------
  Basin Definition: basin
    Cascading Flow: cascade
  Time Series Data: obs, water_use_read, dynamic_param_read
   Potet Solar Rad: soltab
  Temperature Dist: temp_1sta, temp_laps, temp_dist2, climate_hru,
                    temp_map
       Precip Dist: precip_1sta, precip_laps, precip_dist2,
                    climate_hru, precip_map
Temp & Precip Dist: xyz_dist, ide_dist
    Solar Rad Dist: ccsolrad, ddsolrad, climate_hru
Transpiration Dist: transp_tindex, climate_hru, transp_frost
      Potential ET: potet_hamon, potet_jh, potet_pan, climate_hru,
                    potet_hs, potet_pt, potet_pm, potet_pm_sta
      Interception: intcp
Snow & Glacr Dynam: snowcomp, glacr_melt
    Surface Runoff: srunoff_smidx, s

The parameter seg_length has been declared inconsistently in the modules stream_temp and routing. The minimum value has been declared as 0.001 and 1.0. The maximum value has been declared as 200000.0 and 100000.0.
 The parameter description has been set as "Bed slope of each segment"and "Surface slope of each segment." The minimum value has been declared as 0.0001 and 0.0000001.














Using Control File: /mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/NHM.control

Using Parameter File: /mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/myparam.param

Using var_init_file: /mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/daily/restart/2021-12-31.restart

Writing var_save_file: /mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/daily/restart/2022-01-04.restart

Writing PRMS Water Budget File: /mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/daily/output/model.out

WARNING, HRU: 914 soil_rechr_max < 0.00001, set to 0.00001

WARNING, HRU: 112485 soil_rechr_max < 0.00001, set to 0.00001

WARNING, HRU: 113708 soil_rechr_max < 0.00001, set to 0.00001



         set in the Parameter File. Module default values are being used.


 WARNING, seg_slope < 0.0000001, set to 0.0001         188   0.00000000    
 WARNING, seg_slope < 0.0000001, set to 0.0001        3822   0.00000000    
 WARNING, seg_slope < 0.0000001, set to 0.0001        5216   0.00000000    
 WARNING, seg_slope < 0.0000001, set to 0.0001        5671   0.00000000    
 WARNING, seg_slope < 0.0000001, set to 0.0001        7629   0.00000000    
 WARNING, seg_slope < 0.0000001, set to 0.0001        8675   0.00000000    
 WARNING, seg_slope < 0.0000001, set to 0.0001        9679   0.00000000    
 WARNING, seg_slope < 0.0000001, set to 0.0001       11463   0.00000000    
 WARNING, seg_slope < 0.0000001, set to 0.0001       15020   0.00000000    
 WARNING, seg_slope < 0.0000001, set to 0.0001       15760   0.00000000    
 WARNING, seg_slope < 0.0000001, set to 0.0001       15873   0.00000000    
 WARNING, seg_slope < 0.0000001, set to 0.0001       16437   0.00000000    
 WARNING, seg_slope < 0.0000001, set to 0.0001       16740   0.00000000    
 WARNING, se

CompletedProcess(args=['/home/rmcd/prms_5.2.1_linux/bin/prms', '-set', 'start_time', '2022,1,1,0,0,0', '-set', 'end_time', '2022,1,4,0,0,0', '-set', 'humidity_day', '/mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/daily/input/humidity.cbh', '-set', 'precip_day', '/mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/daily/input/prcp.cbh', '-set', 'tmax_day', '/mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/daily/input/tmax.cbh', '-set', 'tmin_day', '/mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/daily/input/tmin.cbh', '-set', 'init_vars_from_file', '1', '-set', 'var_init_file', '/mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/daily/restart/2021-12-31.restart', '-set', 'save_vars_to_file', '1', '-set', 'var_save_file', '/mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/daily/restart/2022-01-04.restart', '-set', 'nhruOutBaseFileName', PosixPath('/mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/daily/output'), '-set', 'nsegmentOutBaseFileName', PosixPath('/mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/daily/output'), '-set', 'model_output_file', PosixPath('/mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/daily/output/mo